In [9]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import times

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [18]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/abby/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [19]:
url = 'https://www.walmart.com/search/?query=room%20air%20purifier'
browser.visit(url)

In [20]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [28]:
for product in products:

    link = product.find('a')
    href = link['href']
    product_link = 'https://www.walmart.com' + href
    print(product_link)

    img = product.find('img')['src']
    print(img)

    product_title = product.find('img')['alt']
    print(product_title)

    stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
    print(stars_review)
    
    review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
    print(f"{review_amount} reviews")

    price = product.find('span', class_=s'price-main')
    current_price = price.find('span', class_='visuallyhidden').text
    print(current_price)

#     browser.click_link_by_partial_href(href)

#     browser.quit()


https://www.walmart.com/ip/Filtrete-by-3M-Room-Air-Purifier-Console-110-Sq-ft-coverage-Black-HEPA-Type-Allergen-Filter-Included/475820314
https://i5.walmartimages.com/asr/eae28c2f-5dbe-4a96-b30c-c381a64b056e_1.572259f55119e41bb4d69602667a85b0.jpeg?odnHeight=200&odnWidth=200&odnBg=ffffff
Filtrete by 3M Room Air Purifier, Console, 110 Sq ft coverage, Black, HEPA-Type Allergen Filter Included
4.5
170 reviews
$49.94
https://www.walmart.com/ip/Holmes-Personal-Space-2-Speed-Air-Purifier-with-Air-Ionizer/21930737
https://i5.walmartimages.com/asr/ca7e3b65-d3c8-4a77-9d6a-374597c5721b_1.848425fbf63f21552e8afc8a06855be7.jpeg?odnHeight=200&odnWidth=200&odnBg=ffffff
Holmes Personal Space 2-Speed Air Purifier with Air Ionizer
4.3
485 reviews
$14.98
https://www.walmart.com/ip/Honeywell-HEPA-Allergen-Remover-Air-Purifier-Reduces-up-to-99-9-of-certain-viruses-bacteria-and-mold-spores-HPA300-Black/37770927
https://i5.walmartimages.com/asr/3574cc6b-93c9-4c02-89c2-6756d204fe09_1.7e49f59e7cbf1e7f786efa3a5d

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
review = soup.find('div', class_='CustomerReviews-list')
print(review)

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")